In [1]:
from spark_init import start_spark
spark = start_spark()
spark.range(10).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [3]:
staticDataFrame = spark.read\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv("../data/retail-data/by-day/*.csv")

staticSchema = staticDataFrame.schema


In [5]:
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("../data/retail-data/by-day/*.csv")


In [7]:
from pyspark.sql.functions import window

purchaseByCustomerPerHour = streamingDataFrame\
    .selectExpr("CustomerID",
                ("UnitPrice * Quantity AS total_cost"),
                "InvoiceDate")\
    .groupBy(
        "CustomerID",
        window("InvoiceDate", "1 hour"))\
    .sum("total_cost")\

purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()\
    

In [ ]:
spark.sql("""
    SELECT CAST(CustomerID AS INT) AS CustomerID,
           window,
           `sum(total_cost)`
    FROM customer_purchases
    WHERE CustomerID IS NOT NULL
    ORDER BY `sum(total_cost)` DESC
    """)\
.show(5)


+----------+--------------------+------------------+
|CustomerID|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|     18102|{2010-12-07 16:00...|          25920.37|
|     15749|{2011-01-11 12:00...|           22998.4|
|     14646|{2011-02-21 10:00...|18279.479999999996|
|     14646|{2011-03-29 11:00...|           18247.5|
|     14156|{2011-01-14 14:00...|16774.719999999998|
+----------+--------------------+------------------+
only showing top 5 rows


In [13]:
spark.sql("""
    SELECT * FROM customer_purchases
""")\
.count()

21385

In [ ]:
purchaseByCustomerPerHour.writeStream\
    .format("cpmsole")\
    .queryName("customer_purchases_2")\
    .outputMode("complete")\
    .start()

In [16]:
spark.sql("""
    SELECT * FROM customer_purchases_2
""")\
.count()

0